<a href="https://colab.research.google.com/github/tuhinmallick/AI-for-Fashion/blob/main/Full_Fine_tuning_with_GaLore_on_a_Consumer_GPU_Example_with_Mistral_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows how to fully fine-tune an LLM with GaLore. Mistral 7B is used for example.

All the code is explained in this article:

[GaLore: Full Fine-tuning on Your GPU](https://kaitchup.substack.com/p/galore-full-fine-tuning-on-your-gpu)

*Note: When I wrote this notebook, the layerwise implementation of GaLore didn't work but I left the code so that it's ready to be used when it will be fixed by Hugging Face. The non-layerwise implementation works. *

GPU memory consumptions:
- Layerwise GaLore: 22.5 GB
- Non-layerwsie GaLore: 35 GB


First, we need all these dependencies:

In [ ]:

!pip install -q -U bitsandbytes
!pip install git+https://github.com/jiaweizzhao/GaLore
!pip install -- upgrade transformers
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.7 MB/s eta 0:00:00
  Cloning https://github.com/jiaweizzhao/GaLore to /tmp/pip-req-build-afhvdzej
  Running command git clone --filter=blob:none --quiet https://github.com/jiaweizzhao/GaLore /tmp/pip-req-build-afhvdzej
  Resolved https://github.com/jiaweizzhao/GaLore to commit 1b36c33782bdd74a4d6a4f51bc626ef67f51011f
  Preparing metadata (setup.py) ... done
  Created wheel for galore-torch: filename=galore_torch-1.0-py3-none-any.whl size=13310 sha256=095b2b697c27438e50a222d4bc477332fe9a5f0203528b007be0860dd9a5d48e
  Stored in directory: /tmp/pip-ephem-wheel-cache-rgcgyj_1/wheels/88/47/b5/ca5f75e9f8a2eef76440b7070f8e82f0099831c3d13ebbe221
Successfully built galore-torch
ERROR: Could not find a version that satisfies the requirement upgrade (from versions: none)
ERROR: No matching distribution found for upgrade
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Import all the necessary packages.

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
)
from trl import SFTTrainer

Use bfloat16 and Flash Attention is your GPU is compatible:

In [ ]:
major_version, minor_version = torch.cuda.get_device_capability()
if major_version >= 8:
  !pip install flash-attn
  torch_dtype = torch.bfloat16
  attn_implementation='flash_attention_2'
  print("Your GPU is compatible with FlashAttention and bfloat16.")
else:
  torch_dtype = torch.float16
  attn_implementation='eager'
  print("Your GPU is not compatible with FlashAttention and bfloat16.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 32.7 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.8-cp310-cp310-linux_x86_64.whl size=120853537 sha256=53979129f883680327bf5d13027cd014e2d054f4fb5b8856916686ae315e57d6
  Stored in directory: /root/.cache/pip/wheels/9b/5b/2b/dea8af4e954161c49ef1941938afcd91bb93689371ed12a226
Successfully built flash-attn
Your GPU is compatible with FlashAttention and bfloat16.


Load and configure the tokenizer for fine-tuning:

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left' #Necessary for FlashAttention compatibility

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Load an instruction dataset for fine-tuning:

In [ ]:
ds = load_dataset("timdettmers/openassistant-guanaco")

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/9846 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/518 [00:00<?, ? examples/s]

Load the model and enable gradient checkpointing to reduce memory consumption

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
          model_name, device_map={"": 0},  attn_implementation=attn_implementation, torch_dtype=torch_dtype
)
model.gradient_checkpointing_enable()

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

The following cell configures the training argument for GaLore with layerwise updates. At the time of writing this notebook, this functionnality didn't seem to work.

Expect a consumption of 22.5 GB of GPU RAM.

In [ ]:
training_arguments = TrainingArguments(
        output_dir="./galore_adamw_8bit_layerwise_r512_1e-5_wl/",
        evaluation_strategy="steps",
        do_eval=True,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        log_level="debug",
        optim="galore_adamw_8bit_layerwise",
        optim_args="rank=512, update_proj_gap=200, scale=1.8",
        optim_target_modules=[r".*attn.*", r".*mlp.*"],
        save_strategy = 'epoch',
        logging_steps=50,
        learning_rate=1e-5,
        eval_steps=50,
        fp16= not torch.cuda.is_bf16_supported(),
        bf16= torch.cuda.is_bf16_supported(),
        num_train_epochs=3,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
)

Start training:

In [ ]:
trainer = SFTTrainer(
        model=model,
        train_dataset=ds['train'],
        eval_dataset=ds['test'],
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()

This is another configuration disabling the layerwise updates. It works but consumes 35 GB of GPU RAM.

In [ ]:
training_arguments = TrainingArguments(
        output_dir="./galore_adamw_8bit_r512_1e-5_nol/",
        evaluation_strategy="steps",
        do_eval=True,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        log_level="debug",
        optim="galore_adamw_8bit",
        optim_args="rank=512, update_proj_gap=200, scale=1.8",
        optim_target_modules=[r".*attn.*", r".*mlp.*"],
        save_strategy = 'epoch',
        logging_steps=50,
        learning_rate=1e-5,
        eval_steps=50,
        fp16= not torch.cuda.is_bf16_supported(),
        bf16= torch.cuda.is_bf16_supported(),
        num_train_epochs=3,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
)

trainer = SFTTrainer(
        model=model,
        train_dataset=ds['train'],
        eval_dataset=ds['test'],
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()